In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from category_encoders.ordinal import OrdinalEncoder
from collections import defaultdict
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

from vecstack import stacking, StackingTransformer

from sklearn.metrics import log_loss,accuracy_score

from tensorflow.keras import utils

# pycaret으로 직업 예측 진행

# 데이터 전처리

In [2]:
train = pd.read_csv('./data/train_occpy_pred_final.csv')
test = pd.read_csv('./data/test_occpy_pred_final.csv')

In [3]:
train.head()

,Unnamed: 0,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,Accountants,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0,2.0


In [4]:
train['age'] = train.DAYS_BIRTH.apply(lambda x : -x // 365)
train.DAYS_EMPLOYED = (-1) * train.DAYS_EMPLOYED 
train.loc[(train.DAYS_EMPLOYED < 0), 'DAYS_EMPLOYED'] = 0
train.begin_month = (-1) * train.begin_month
# train['personal_id'] = train['gender'] + "_" + train['DAYS_BIRTH'].astype(str) + "_" + train['income_total'].astype(str) + "_" + train['income_type'].astype(str)
# train.gender = train.gender.replace({'F' : 0, 'M' : 1})
# train.car = train.car.replace({'N' : 0, 'Y' : 1})
# train.reality = train.reality.replace({'N' : 0, 'Y' : 1})

train = train.drop(['Unnamed: 0','DAYS_BIRTH'], axis = 1)
train.head(3)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit,age
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,0,0,0,Accountants,2.0,6.0,1.0,38
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,0,0,1,Laborers,3.0,5.0,1.0,31
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,4434,0,1,0,Managers,2.0,22.0,2.0,52


In [5]:
test['age'] = test.DAYS_BIRTH.apply(lambda x : -x // 365)
test.DAYS_EMPLOYED = (-1) * test.DAYS_EMPLOYED 
test.loc[(test.DAYS_EMPLOYED < 0), 'DAYS_EMPLOYED'] = 0
test.begin_month = (-1) * test.begin_month
# test['personal_id'] = test['gender'] + "_" + test['DAYS_BIRTH'].astype(str) + "_" + test['income_total'].astype(str) + "_" + test['income_type'].astype(str)
# test.gender = test.gender.replace({'F' : 0, 'M' : 1})
# test.car = test.car.replace({'N' : 0, 'Y' : 1})
# test.reality = test.reality.replace({'N' : 0, 'Y' : 1})

test = test.drop(['Unnamed: 0','DAYS_BIRTH'], axis = 1)
test.head(3)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,age
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,0,0,1,0,Security staff,2.0,60.0,60
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,8671,0,1,0,Core staff,2.0,36.0,51
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,217,1,1,0,Laborers,2.0,40.0,43


In [6]:
y = train.credit
X = train.drop(['credit'], axis = 1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## scaled_X_train, y_train 으로 학습
## scaled_X_test, y_test 로 예측

In [9]:
numeric_features = ['child_num', 'income_total', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'age']
categorical_features = ['gender','car','reality','income_type','edu_type','family_type','house_type','work_phone','phone','email','occyp_type']

numeric_transformer = StandardScaler()
numeric_transformer.fit(X_train[numeric_features])
X_train[numeric_features] = numeric_transformer.transform(X_train[numeric_features])
X_test[numeric_features] = numeric_transformer.transform(X_test[numeric_features])

In [10]:
clf_catboost = CatBoostClassifier(iterations=500, random_state=123, early_stopping_rounds=50)
clf_catboost.fit(X_train, y_train, 
                 cat_features=categorical_features, verbose=0)

pred_proba = clf_catboost.predict_proba(X_test)
log_loss(y_test, pred_proba)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

ERROR! Session/line number was not unique in database. History logging moved to new session 485


0.765699950649888

### 전체 데이터로 학습 -> 데이콘 제출

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv

In [16]:
train_y = train.credit
train_X = train.drop('credit', axis = 1)
test_X = test

In [17]:
numeric_features = ['child_num', 'income_total', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'age']
categorical_features = ['gender','car','reality','income_type','edu_type','family_type','house_type','work_phone','phone','email','occyp_type']

numeric_transformer = StandardScaler()
numeric_transformer.fit(X_train[numeric_features])
train_X[numeric_features] = numeric_transformer.transform(train_X[numeric_features])
test_X[numeric_features] = numeric_transformer.transform(test_X[numeric_features])

In [18]:
clf_catboost = CatBoostClassifier(iterations=500, random_state=123, early_stopping_rounds=50)
clf_catboost.fit(train_X, train_y, 
                 cat_features=categorical_features, verbose=0)

pred_proba = clf_catboost.predict_proba(test_X)

In [19]:
submission = pd.read_csv('./data/sample_submission.csv')
submission.loc[:, 1:] = pred_proba
submission

,index,0,1,2
0,26457,0.038912,0.119836,0.841252
1,26458,0.138120,0.130527,0.731353
2,26459,0.087454,0.083996,0.828550
3,26460,0.185575,0.104015,0.710411
4,26461,0.136003,0.201789,0.662209
...,...,...,...,...
9995,36452,0.144568,0.227455,0.627978
9996,36453,0.095697,0.307315,0.596989
9997,36454,0.037065,0.067375,0.895560
9998,36455,0.072639,0.156135,0.771226


In [20]:
submission.to_csv('./data/submission_CATBOOST.csv', index = False)

### Stacking

In [16]:
xgb_clf = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
               missing=np.nan, monotone_constraints='()', n_estimators=170,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=10)

lgbm_clf = LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=6, min_child_weight=0.001, min_split_gain=0.6,
                n_estimators=100, n_jobs=-1, num_leaves=30, objective=None,
                random_state=42, reg_alpha=0.001, reg_lambda=5, silent='warn',
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

gb_clf = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=8,
                            max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.001, min_impurity_split=None,
                            min_samples_leaf=5, min_samples_split=10,
                            min_weight_fraction_leaf=0.0, n_estimators=290,
                            n_iter_no_change=None, 
                            random_state=42, subsample=0.25, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [18]:
models = [xgb_clf, lgbm_clf, gb_clf]

S_train, S_test = stacking(models, 
                       scaled_X_train, y_train, scaled_X_test, 
                       regression = False, 
                       metric = accuracy_score,
#                        needs_proba = True,                           
                       n_folds = 5, stratified = True, shuffle = True, 
                       random_state = 0, verbose = 1)

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [XGBClassifier]
[09:36:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:36:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:37:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:37:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[09:37:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:37:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:37:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:37:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:38:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:38:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
    ----
    MEAN:     [0.69964564] + [0.00527104]
    FULL:     [0.69964564]

model  1:     [LGBMClassifier]
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_

In [19]:
xgb_final = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
               missing=np.nan, monotone_constraints='()', n_estimators=170,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=10)

In [20]:
xgb_final.fit(S_train, y_train)
pred_proba = xgb_final.predict_proba(S_test)


[09:40:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:40:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [21]:
pred = xgb_final.predict(S_test)
print(accuracy_score(y_test, pred))

0.7116402116402116


In [22]:
log_loss(y_test, pred_proba)

0.7792928575207947

In [23]:
train_y = train.credit
train_X = train.drop('credit', axis = 1)

In [24]:
numeric_features = ['child_num', 'income_total', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'age']
numeric_transformer = StandardScaler()

categorical_features = ['income_type', 'edu_type', 'family_type', 'house_type','occyp_type']
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown = 'ignore')


preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features)
                ], remainder='passthrough'
                )

In [25]:
preprocessor.fit(train_X)
scaled_train = preprocessor.transform(train_X)
scaled_test = preprocessor.transform(test)

In [26]:
xgb_clf = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
               missing=np.nan, monotone_constraints='()', n_estimators=170,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=10)

lgbm_clf = LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=6, min_child_weight=0.001, min_split_gain=0.6,
                n_estimators=100, n_jobs=-1, num_leaves=30, objective=None,
                random_state=42, reg_alpha=0.001, reg_lambda=5, silent='warn',
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

gb_clf = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=8,
                            max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.001, min_impurity_split=None,
                            min_samples_leaf=5, min_samples_split=10,
                            min_weight_fraction_leaf=0.0, n_estimators=290,
                            n_iter_no_change=None, 
                            random_state=42, subsample=0.25, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [39]:
estimators = [
    ('XGB', xgb_clf),
    ('LGBM', lgbm_clf),
    ('GB', gb_clf)
    ]

In [40]:
stack = StackingTransformer(estimators,
                       regression = False, 
                       metric = accuracy_score,
#                        needs_proba = True,                           
                       n_folds = 5, stratified = True, shuffle = True, 
                       random_state = 0, verbose = 1)

In [41]:
stack.fit(scaled_train, train_y)

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
variant:      [A]
n_estimators: [3]

estimator  0: [XGB: XGBClassifier]
[15:29:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:29:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:29:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:29:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:30:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:30:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:30:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:30:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:30:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_cat_to_onehot" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:30:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
    MEAN:     [0.70586246] + [0.00285516]

estimator  1: [LGBM: LGBMClassifier]
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample

StackingTransformer(estimators=[('XGB',
                                 XGBClassifier(base_score=0.5, booster='gbtree',
                                               callbacks=None,
                                               colsample_bylevel=1,
                                               colsample_bynode=1,
                                               colsample_bytree=0.7,
                                               early_stopping_rounds=None,
                                               enable_categorical=False,
                                               eval_metric=None, gamma=0,
                                               gpu_id=-1,
                                               grow_policy='depthwise',
                                               importance_type=None,
                                               interaction_constraints='',
                                               learning_rate=0.3, max_bin=256,
                                     

In [42]:
S_train = stack.transform(scaled_train)
S_test = stack.transform(scaled_test)

Train set was detected.
Transforming...

estimator  0: [XGB: XGBClassifier]
    DONE

estimator  1: [LGBM: LGBMClassifier]
    DONE

estimator  2: [GB: GradientBoostingClassifier]
    DONE

Transforming...

estimator  0: [XGB: XGBClassifier]
    DONE

estimator  1: [LGBM: LGBMClassifier]
    DONE

estimator  2: [GB: GradientBoostingClassifier]
    DONE



In [44]:
model = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3, eval_metric='mlogloss') 
model = model.fit(S_train, train_y) 

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [45]:
y_pred = model.predict(S_test) 
print(y_pred)

[2. 2. 2. ... 2. 2. 2.]


In [49]:
y_pred_proba = model.predict_proba(S_test) 
print(y_pred_proba)

[[0.10300916 0.17039236 0.72659844]
 [0.10300916 0.17039236 0.72659844]
 [0.10300916 0.17039236 0.72659844]
 ...
 [0.10300916 0.17039236 0.72659844]
 [0.10300916 0.17039236 0.72659844]
 [0.10300916 0.17039236 0.72659844]]


In [55]:
submission = pd.read_csv('./data/sample_submission.csv')
submission.loc[:, 1:] = y_pred_proba
submission
submission.to_csv('./data/submission.csv', index = False)

<ipython-input-55-1d42f7cf5abb>:2: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  submission.loc[:, 1:] = y_pred_proba


,index,0,1,2
0,26457,0.103009,0.170392,0.726598
1,26458,0.103009,0.170392,0.726598
2,26459,0.103009,0.170392,0.726598
3,26460,0.103009,0.170392,0.726598
4,26461,0.103009,0.170392,0.726598
...,...,...,...,...
9995,36452,0.103009,0.170392,0.726598
9996,36453,0.103009,0.170392,0.726598
9997,36454,0.103009,0.170392,0.726598
9998,36455,0.103009,0.170392,0.726598


In [57]:
submission.to_csv('./data/submission.csv', index = False)